In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

train = pd.read_csv("data/raw/Restaurants/train.csv")
test = pd.read_csv("data/raw/Restaurants/test_x.csv")
sample = pd.read_csv("data/raw/Restaurants/sample_submission.csv")

train["test"] = train["rating"] * (0.9 + train["craft_beer"] * 0.2) * (0.9 + train["wi_fi"] * 0.15) * (0.95 + train["special_menu"] * 0.1) * (0.975 + train["car_park"] * 0.05)
train["test"] = (train["test"] - train["test"].mean()) / (train["test"].max() - train["test"].min())
test["test"] = test["rating"] * (0.9 + test["craft_beer"] * 0.2) * (0.9 + test["wi_fi"] * 0.15)* (0.95 + test["special_menu"] * 0.1) * (0.975 + test["car_park"] * 0.05)
test["test"] = (test["test"] - test["test"].mean()) / (test["test"].max() - test["test"].min())
# test["test"]


train["rubrics"] = train["rubrics"].map({
    "Ресторан" : 0,
    "Кафе" : 1,
    "Кофейня" : 2,
    "Кондитерская" : 3,
    "Быстрое питание" : 4,
    "Бар, паб" : 5,
    "Кальян-бар" : 6,
    "Булочная, пекарня" : 7,
    "Спортбар" : 8,
    "Столовая" : 9,
    "Пиццерия" : 10,
    "Суши-бар" : 11,
    "Кофе с собой" : 12,
    "Бар безалкогольных напитков" : 13
})
test["rubrics"] = test["rubrics"].map({
    "Ресторан" : 0,
    "Кафе" : 1,
    "Кофейня" : 2,
    "Кондитерская" : 3,
    "Быстрое питание" : 4,
    "Бар, паб" : 5,
    "Кальян-бар" : 6,
    "Булочная, пекарня" : 7,
    "Спортбар" : 8,
    "Столовая" : 9,
    "Пиццерия" : 10,
    "Суши-бар" : 11,
    "Кофе с собой" : 12,
    "Бар безалкогольных напитков" : 13
})


train["user_city"] = train["user_city"].map({"spb" : 0,"msk" : 1})
test["user_city"] = test["user_city"].map({
    "spb" : 0,
    "msk" : 1
})
train["org_city"] = train["org_city"].map({
    "spb" : 0,
    "msk" : 1
})
test["org_city"] = test["org_city"].map({
    "spb" : 0,
    "msk" : 1
})

train.drop(['user_id', "average_bill", "automatic_door", "closed_for_quarantine", "elevator_wheelchair_accessible", "wheelchair_accessible", "wheelchair_access", "parking_disabled", "toilet_for_disabled"], axis=1, inplace=True)
test.drop(['user_id', "average_bill", "automatic_door", "closed_for_quarantine", "elevator_wheelchair_accessible", "wheelchair_accessible", "wheelchair_access", "parking_disabled", "toilet_for_disabled"], axis=1, inplace=True)
# test["rubrics"]
train

,org_id,rating,ts,user_city,org_city,rating_org,rubrics,food_delivery,breakfast,takeaway,...,call_button,table_games,vinotheque,cakes_for_different_occasions,accepted_credit_cards,kalyan,teahouse,bread_from_tandoor,handmade_goods,test
0,11946401375069641937,4.0,677,1,1,3.883212,0,1,1,1,...,0,0,0,0,0,0,0,0,0,-0.038254
1,5545859981678495568,5.0,577,1,1,4.495379,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0.232432
2,3451485117888180206,4.0,716,1,1,4.576063,1,1,1,1,...,0,0,0,0,0,0,0,0,0,-0.105691
3,9490041040735762889,3.0,639,0,0,3.775701,1,1,1,1,...,0,0,0,0,0,0,0,0,0,-0.097261
4,5234254278902729372,5.0,669,1,1,4.187500,10,1,1,1,...,0,0,0,0,0,0,0,0,0,0.138769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73990,2948132663120061820,2.0,517,1,1,4.280632,7,1,1,0,...,0,0,0,0,0,0,0,0,0,-0.426018
73991,13225228120465186798,5.0,455,1,1,4.446033,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.054472
73992,1135274990644701922,5.0,1032,1,1,4.420231,5,1,1,1,...,0,0,0,0,0,0,0,0,0,0.335461
73993,2498968754330575170,5.0,27,1,1,4.694307,1,0,0,0,...,0,0,0,0,0,0,0,0,0,-0.059930


In [2]:
# test["org_city"].unique()
# train.isna().sum()
# test1
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73995 entries, 0 to 73994
Data columns (total 64 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   org_id                         73995 non-null  uint64 
 1   rating                         73995 non-null  float64
 2   ts                             73995 non-null  int64  
 3   user_city                      73995 non-null  int64  
 4   org_city                       73995 non-null  int64  
 5   rating_org                     73995 non-null  float64
 6   rubrics                        73995 non-null  int64  
 7   food_delivery                  73995 non-null  int64  
 8   breakfast                      73995 non-null  int64  
 9   takeaway                       73995 non-null  int64  
 10  summer_terrace                 73995 non-null  int64  
 11  wi_fi                          73995 non-null  int64  
 12  business_lunch                 73995 non-null 

In [3]:
scaler = StandardScaler()
scaler2 = MinMaxScaler()

# print(scaler.fit_transform(train[["rubrics"]]))
# train["org_id"] = scaler2.fit_transform(train[["org_id"]])
# test["org_id"] = scaler2.fit_transform(test[["org_id"]])
#
# train["rubrics"] = scaler.fit_transform(train[["rubrics"]])
# test["rubrics"] = scaler.fit_transform(test[["rubrics"]])

# train["org_id"] = scaler.fit_transform(train["org_id"])
# test["org_id"] = scaler.fit_transform(test["org_id"])



In [4]:

def feature_regressor(df, feature, k):

    # np.random.seed(1)
    # shuffled_index = np.random.permutation(df.index)
    # rand_df = df.reindex(shuffled_index)
    # x_train, x_test, y_train, y_test = train_test_split(df.drop(["rating_org"], axis=1), df["rating_org"], test_size=0.25, random_state=4)
    x_train, x_test, y_train, y_test = train_test_split(df[["rating", "rubrics", "org_id", "wi_fi", "test", "org_city"]], df["rating_org"], test_size=0.25, random_state=4)

    # Split the dataset
    # train_set = rand_df.iloc[0:int(len(rand_df)/2)]
    # test_set = rand_df.iloc[int(len(rand_df)/2):]

    # Train
    knn = KNeighborsRegressor(n_neighbors=k)
    knn.fit(x_train, y_train)
    # knn.fit(train_set[[feature]], train_set["rating_org"])


    # Predict
    # predictions = knn.predict(test_set[[feature]])
    predictions = knn.predict(x_test)
    # return mean_absolute_percentage_error(test_set["rating_org"], predictions)
    return mean_absolute_percentage_error(y_test, predictions)



In [5]:
feature_regressor(train, "rating", 25)

0.04391528846586088

In [6]:
# print(27, feature_regressor(train, "rating", 500))
# print(27, feature_regressor(train, "rating", 2500))
# print(27, feature_regressor(train, "rating", 10000))


In [7]:

features = ["test"]

res = []
for feature in features:
    for i in range(1, 30):
        err = feature_regressor(train, feature, i)
        print(err)
        res.append((i, err))
    #res.append((500, feature_regressor(train, feature, 500)))
    #res.append((1500, feature_regressor(train, feature, 1500)))
print("Complete")

0.017452150091277707
0.021980983037264906
0.024852074499029736
0.027428474828210852
0.02935599850656689
0.031235145957447447
0.03266834324520416
0.03388124631769941
0.034994475991935316
0.03603446318324439
0.037065510779104374
0.03795438518000051
0.0386688338321457
0.039226578284648234
0.03980323018522754
0.040383796900624096
0.0409616004660244
0.04148958410330627
0.04199107937230593
0.04240878972896726
0.04278743983179952
0.04311356009967245
0.04340374025591899
0.043668248371458364
0.04391528846586088
0.04418998010253196
0.044463334546224954
0.04471684865656024
0.04492018401681466
Complete


In [8]:
print(sorted(res, key=lambda x: x[1]))

[(1, 0.017452150091277707), (2, 0.021980983037264906), (3, 0.024852074499029736), (4, 0.027428474828210852), (5, 0.02935599850656689), (6, 0.031235145957447447), (7, 0.03266834324520416), (8, 0.03388124631769941), (9, 0.034994475991935316), (10, 0.03603446318324439), (11, 0.037065510779104374), (12, 0.03795438518000051), (13, 0.0386688338321457), (14, 0.039226578284648234), (15, 0.03980323018522754), (16, 0.040383796900624096), (17, 0.0409616004660244), (18, 0.04148958410330627), (19, 0.04199107937230593), (20, 0.04240878972896726), (21, 0.04278743983179952), (22, 0.04311356009967245), (23, 0.04340374025591899), (24, 0.043668248371458364), (25, 0.04391528846586088), (26, 0.04418998010253196), (27, 0.044463334546224954), (28, 0.04471684865656024), (29, 0.04492018401681466)]


In [20]:
def get_result(train, test, feature, k):
    # np.random.seed(1)
    # shuffled_index = np.random.permutation(train.index)
    # rand_df = train.reindex(shuffled_index)
    # x_train, x_test, y_train, y_test = train_test_split(train.drop(["rating_org"], axis=1), train["rating_org"], test_size=0.0, random_state=4)
    # x_train, x_test, y_train, y_test = train_test_split(train.drop(["rating_org"], axis=1), train["rating_org"], test_size=0.0, random_state=4)
    # x_train
    # Split the dataset
    # train_set = rand_df.iloc[0:int(len(rand_df)/2)]
    # test_set = rand_df.iloc[int(len(rand_df)/2):]

    # Train
    # rand_df.head()
    # train_set = rand_df.iloc[:]
    # train_set.head()
    knn = KNeighborsRegressor(n_neighbors=k)
    # train = train.sort_values(by=["rubrics"])
    # knn.fit(train[["rating", "org_id", "rubrics"]], train["rating_org"])
    # knn.fit(train[["rating", "org_id", "rubrics", "test"]], train["rating_org"])
    knn.fit(train[["rating", "org_id", "rubrics", "wi_fi", "test", "org_city"]], train["rating_org"])

    # knn.fit(train.drop(["rating_org"], axis=1), train["rating_org"])
    # knn.fit(train_set.drop(["rating_org"], axis=1), train_set["rating_org"])
    # knn.fit(train_set[[feature]], train_set["rating_org"])


    # Predict
    # predictions = knn.predict(test_set[[feature]])
    # predictions = knn.predict(test[["rating", "org_id", "rubrics"]])
    # predictions = knn.predict(test[["rating", "org_id", "rubrics", "test"]])
    predictions = knn.predict(test[["rating", "org_id", "rubrics", "wi_fi", "test", "org_city"]])
    # return mean_absolute_percentage_error(test_set["rating_org"], predictions)
    # return mean_absolute_percentage_error(y_test, predictions)
    # return []

    return predictions

predictions = get_result(train, test, "test", 10000)
print(predictions)

[4.30302351 4.32932965 4.33827407 ... 4.29213019 4.30949172 4.33359029]


In [10]:
# train = train.sort_values(by=["rubrics"])

In [19]:
next = np.column_stack((np.arange(0, len(predictions)), predictions))
# print(next)
out = pd.DataFrame(next, columns=["id", "rating_org"])
out["id"] = out["id"].astype(int)
out.to_csv("data/processed/Restaurants/predicts.csv", index=False)
print("saved")

saved


sort by id org, type
